In [ ]:
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [ ]:
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

In [ ]:
DATA_DIR = "DATA_DIR"
WORK_DIR = "WORK_DIR"
os.environ['DATA_DIR'] = DATA_DIR

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

! wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
! unzip -o SST-2.zip -d {DATA_DIR}

In [ ]:
! sed 1d {DATA_DIR}/SST-2/train.tsv > {DATA_DIR}/SST-2/train_nemo_format.tsv
! sed 1d {DATA_DIR}/SST-2/dev.tsv > {DATA_DIR}/SST-2/dev_nemo_format.tsv
! ls -l {DATA_DIR}/SST-2

In [ ]:
# let's take a look at the data 
print('Contents (first 5 lines) of train.tsv:')
! head -n 5 {DATA_DIR}/SST-2/train_nemo_format.tsv

print('\nContents (first 5 lines) of test.tsv:')
! head -n 5 {DATA_DIR}/SST-2/test.tsv

In [ ]:
MODEL_CONFIG = "text_classification_config.yaml"
CONFIG_DIR = WORK_DIR + '/configs/'

os.makedirs(CONFIG_DIR, exist_ok=True)
if not os.path.exists(CONFIG_DIR + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + MODEL_CONFIG, CONFIG_DIR)
    print('Config file downloaded!')
else:
    print ('config file already exists')
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)

In [ ]:
config.model.dataset.num_classes=2

In [ ]:
config.model.train_ds.file_path = os.path.join(DATA_DIR, 'SST-2/train_nemo_format.tsv')
config.model.validation_ds.file_path = os.path.join(DATA_DIR, 'SST-2/dev_nemo_format.tsv')

config.save_to = 'trained-model.nemo'
config.export_to = 'trained-model.onnx'

print("Train dataloader's config: \n")
print(OmegaConf.to_yaml(config.model.train_ds))

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
config.trainer.accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.strategy = None
config.trainer.max_epochs = 1
trainer = pl.Trainer(**config.trainer)

In [ ]:
trainer = pl.Trainer(**config.trainer)
print(OmegaConf.to_yaml(config.exp_manager))
exp_dir = exp_manager(trainer, config.exp_manager)
print(exp_dir)

In [ ]:
print(nemo_nlp.modules.get_pretrained_lm_models_list())

In [ ]:
config.model.language_model.pretrained_model_name = "bert-base-uncased"

In [ ]:
model = nemo_nlp.models.TextClassificationModel(cfg=config.model, trainer=trainer)

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")


In [ ]:
trainer.fit(model)
model.save_to(config.save_to)

In [ ]:
checkpoint_path = trainer.checkpoint_callback.best_model_path
eval_model = nemo_nlp.models.TextClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)
eval_config = OmegaConf.create({'file_path': config.model.validation_ds.file_path, 'batch_size': 64, 'shuffle': False, 'num_samples': -1})
eval_model.setup_test_data(test_data_config=eval_config)

eval_trainer_cfg = config.trainer.copy()
eval_trainer_cfg.accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
eval_trainer_cfg.strategy = None 
eval_trainer = pl.Trainer(**eval_trainer_cfg)

eval_trainer.test(model=eval_model, verbose=False)

In [ ]:
checkpoint_path = trainer.checkpoint_callback.best_model_path
infer_model = nemo_nlp.models.TextClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

In [ ]:
if torch.cuda.is_available():
    infer_model.to("cuda")
else:
    infer_model.to("cpu")

queries = ["We are seeing deaths. So we have had deaths from people who sat in the waiting room and have passed away in the waiting room",
           "We need to email to our local political representatives and tell them that ER services are failing and we need to fix it",
           "It could be you, me or anyone else, who will get to be the first one's to suffer",
           "It could be a baby, it could be an infant, it could be a mother, it could be a father who will loose their life because of ER failure", 
           "It could be a newly wed, or too be wed, it could be a student, a teacher, a farmer, an engineer, doctor or Professor",
           "No matter who you are, what you do, where do you live, what your race is, what your background is. If ER fails we all would be probably victims",
           "Speak up let them know the real issues. Let them know we want change. Let them know we want actions",
           "It seems to me that modern primary, secondary or even tertiary education system is just mass producing conformist enablers who are incapable of looking farther than their immediate sphere of life",
           "They become like horses who are given side blinds to let them focus on a certain path",
           "Why is everyone so conforming/accepting to everything as if whatever is happening is the most ideal/efficient/best/just/moral way to do things?",
           "I am not asking anyone to break the law. I am not advocating anarchy. I am just questioning the attitude of not questioning enough.",
           "I see consumers not knowing their rights, their powers, their unused influence. We are so socially conditioned to accept whatever is going on is good. I see voters oblivious to what they should ask for from candidates.",
           "May be this is so gradual that people don't notice. People are made conformist and then burdened with debt, with responsibilities shoved down upon them by societal expectations. This creates a nation of followers. A follower nation cannot take care of itself. It depends on an active force to be there to guide them to what they think is the best. And this can go wrong too.",
           "We need disruptive mindset. We need to ask questions. Questions such as, do I deserve to be treated this way?"]
           
results = infer_model.classifytext(queries=queries, batch_size=3, max_seq_length=512)

print('The prediction results of some sample queries with the trained model:')
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Predicted label: {result}')


## Training Script

If you have NeMo installed locally (eg. cloned from the Github), you can also train the model with `examples/nlp/text_classification/text_classifciation_with_bert.py`. This script contains an example on how to train, evaluate and perform inference with the TextClassificationModel.

For example the following would train a model for 50 epochs in 2 GPUs on a classification task with 2 classes:

```
# python text_classification_with_bert.py 
        model.dataset.num_classes=2
        model.train_ds=PATH_TO_TRAIN_FILE 
        model.validation_ds=PATH_TO_VAL_FILE  
        trainer.max_epochs=50
        trainer.devices=2
        trainer.accelerator='gpu'
```

This script would also reload the best checkpoint after the training is done and does evaluation on the dev set. Then perform inference on some sample queries. 


By default, this script uses `examples/nlp/text_classification/conf/text_classifciation_config.py` config file, and you may update all the params in the config file from the command line. You may also use another config file like this:

```
# python text_classification_with_bert.py --config-name==PATH_TO_CONFIG_FILE
        model.dataset.num_classes=2
        model.train_ds=PATH_TO_TRAIN_FILE 
        model.validation_ds=PATH_TO_VAL_FILE  
        trainer.max_epochs=50
        trainer.devices=2
        trainer.accelerator='gpu'
```


## Deployment

You can also deploy a model to an inference engine (like TensorRT or ONNXRuntime) using ONNX exporter.
If you don't have one, let's install it:

In [ ]:
!pip install --upgrade onnxruntime # for gpu, use onnxruntime-gpu
# !mkdir -p ort
# %cd ort
# !git clean -xfd
# !git clone --depth 1 --branch v1.8.0 https://github.com/microsoft/onnxruntime.git .
# !./build.sh --skip_tests --config Release --build_shared_lib --parallel --use_cuda --cuda_home /usr/local/cuda --cudnn_home /usr/lib/x86_64-linux-gnu --build_wheel
# !pip uninstall -y onnxruntime
# !pip uninstall -y onnxruntime-gpu
# !pip install  --upgrade --force-reinstall ./build/Linux/Release/dist/onnxruntime*.whl
# %cd ..

Then export

In [ ]:
model.export(config.export_to)

And run some queries

In [ ]:
import numpy as np
import torch
from nemo.utils import logging
from nemo.collections.nlp.parts.utils_funcs import tensor2list
from nemo.collections.nlp.models.text_classification import TextClassificationModel
from nemo.collections.nlp.data.text_classification import TextClassificationDataset

import onnxruntime

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def postprocessing(results, labels):
    return [labels[str(r)] for r in results]

def create_infer_dataloader(model, queries):
    batch_size = len(queries)
    dataset = TextClassificationDataset(tokenizer=model.tokenizer, queries=queries, max_seq_length=512)
    return torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
        drop_last=False,
        collate_fn=dataset.collate_fn,
    )

queries = ["We are seeing deaths. So we have had deaths from people who sat in the waiting room and have passed away in the waiting room",
           "We need to email to our local political representatives and tell them that ER services are failing and we need to fix it",
           "It could be you, me or anyone else, who will get to be the first one's to suffer",
           "It could be a baby, it could be an infant, it could be a mother, it could be a father who will loose their life because of ER failure", 
           "It could be a newly wed, or too be wed, it could be a student, a teacher, a farmer, an engineer, doctor or Professor",
           "No matter who you are, what you do, where do you live, what your race is, what your background is. If ER fails we all would be probably victims",
           "Speak up let them know the real issues. Let them know we want change. Let them know we want actions",
           "It seems to me that modern primary, secondary or even tertiary education system is just mass producing conformist enablers who are incapable of looking farther than their immediate sphere of life",
           "They become like horses who are given side blinds to let them focus on a certain path",
           "Why is everyone so conforming/accepting to everything as if whatever is happening is the most ideal/efficient/best/just/moral way to do things?",
           "I am not asking anyone to break the law. I am not advocating anarchy. I am just questioning the attitude of not questioning enough.",
           "I see consumers not knowing their rights, their powers, their unused influence. We are so socially conditioned to accept whatever is going on is good. I see voters oblivious to what they should ask for from candidates.",
           "May be this is so gradual that people don't notice. People are made conformist and then burdened with debt, with responsibilities shoved down upon them by societal expectations. This creates a nation of followers. A follower nation cannot take care of itself. It depends on an active force to be there to guide them to what they think is the best. And this can go wrong too.",
           "We need disruptive mindset. We need to ask questions. Questions such as, do I deserve to be treated this way?"]

model.eval()
infer_datalayer = create_infer_dataloader(model, queries)

ort_session = onnxruntime.InferenceSession(config.export_to)

for batch in infer_datalayer:
    input_ids, input_type_ids, input_mask, subtokens_mask = batch
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_ids),
                  ort_session.get_inputs()[1].name: to_numpy(input_mask),
                  ort_session.get_inputs()[2].name: to_numpy(input_type_ids),}
    ologits = ort_session.run(None, ort_inputs)
    alogits = np.asarray(ologits)
    logits = torch.from_numpy(alogits[0])
    preds = tensor2list(torch.argmax(logits, dim = -1))
    processed_results = postprocessing(preds, {"0": "negative", "1": "positive"})

    logging.info('The prediction results of some sample queries with the trained model:')
    for query, result in zip(queries, processed_results):
        logging.info(f'Query : {query}')
        logging.info(f'Predicted label: {result}')
    break